In [1]:
import numpy as np
import pandas as pd
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph
from pgmpy.inference import BeliefPropagation

## Task 2 

#### Task 2.1

In [2]:
G = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f = DiscreteFactor(['S1'], [2], [0.85,0.15])
g = DiscreteFactor(['S1', 'E1'], [2, 2], [0.1,0.2,0.0,0.5])

###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G.add_nodes_from(['S1','E1'])  ## Add random variables 
G.add_factors(g)     ## Add factor functions
G.add_factors(f)
###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G.add_edges_from([('S1',f), ('S1',g), ('E1',g)])

#### Task 2.2

In [3]:
bp = BeliefPropagation(G)
###############################
#   TODO: Compute the marginal probability
###############################
P_S1 = bp.query(['S1'])
P_S1.normalize()
print(P_S1)

Eliminating: E1: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.29it/s]

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.7727 |
+-------+-----------+
| S1(1) |    0.2273 |
+-------+-----------+


#### Task 2.3

In [4]:
print("S1=0 maximizes the marginal probability P(S1).")

S1=0 maximizes the marginal probability P(S1).


#### Task 2.4

In [5]:
P_S1_E1 = bp.query(['S1'], evidence={'E1':1})
P_S1_E1.normalize()
print(P_S1_E1)

0it [00:00, ?it/s]

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |    0.6939 |
+-------+-----------+
| S1(1) |    0.3061 |
+-------+-----------+


#### Task 2.5

In [6]:
print("The most probable state when observing E1=1 is S1=0.")

The most probable state when observing E1=1 is S1=0.


#### Task 2.6

## Task 3

In [7]:
ATTACK_EVENTS_MAP = {
    'Scan':1,
    'Login':2,
    'Sensitive_URI':3,
    'New_Kernel_Module':4,
    'DNS_Tunneling':5
}
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}

FACTOR_FUNCTION_MAP = {
    'Scan': [0] * 11,
    'Login': [0] * 11,
    'Sensitive_URI': [0] * 11,
    'New_Kernel_Module': [0] * 11,
    'DNS_Tunneling': [0] * 11
}

SEQUENCE_EVENT_MAP = {'e1': 'Scan',
                      'e2': 'Login',
                      'e3': 'Sensitive_URI',
                      'e4': 'Sensitive_URI',
                      'e5': 'Sensitive_URI',
                      'e6': 'New_Kernel_Module',
                      'e7': 'DNS_Tunneling',
                      'e8': 'DNS_Tunneling',
                      'e9': 'DNS_Tunneling'}

### Task 3.0 

In [8]:
event_review = open('data/event_review.txt', 'r') 
event_reviews = event_review.readlines()

events = ['Scan','Login']+['Sensitive_URI']*3+['New_Kernel_Module']+['DNS_Tunneling']*3
facfuncs = []
for line in event_reviews:
    #print("Line: ", line)
    for attack_event in ATTACK_EVENTS_MAP.keys():
        if attack_event in line:
            #print("   Attack Event: ", attack_event)
            for attack_state in ATTACK_STATES_MAP.keys():
                if attack_state in line:
                    #print("   Attack State: ", attack_state)
                    attack_state_index = ATTACK_STATES_MAP[attack_state]-1
                    FACTOR_FUNCTION_MAP[attack_event][attack_state_index] += 1
                    break
            break
    #FACTOR_FUNCTION_MAP[attack_event][ATTACK_STATES_MAP[attack_state]-1] += 1
for _, function in FACTOR_FUNCTION_MAP.items():
    total = sum(function)
    for i in range(len(function)):
        function[i] = function[i]/total

import pandas as pd
pd.DataFrame(FACTOR_FUNCTION_MAP['DNS_Tunneling']).transpose()

,0,1,2,3,4,5,6,7,8,9,10
0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98,0.0,0.0


### Task 3.1

In [9]:
from collections import defaultdict

attack_events = []
attack_states = []

attack_sequences = open('data/attack_sequences.txt', 'r').readlines()

count = 0
commons = defaultdict(int)

for sequence in attack_sequences:
    sequence = sequence.split(' ')[:-1]
    j = 3
    while j <= len(sequence):
        commons[tuple(sequence[j-3:j])] += 1
        j += 1

repetitives = defaultdict(int)

for key, value in commons.items():
    if key[0] == key[1] and key[1] == key[2]:
        repetitives[key] = value
        
most_common_sequence = max(commons.items(), key=lambda item: item[1])
most_frequent_repetitive_sequence = max(repetitives.items(), key=lambda item: item[1])
print(most_common_sequence)
print(most_frequent_repetitive_sequence)

(('Scan', 'Sensitive_URI', 'New_Kernel_Module'), 200)
(('Sensitive_URI', 'Sensitive_URI', 'Sensitive_URI'), 186)


In [10]:
countc = 0
countr = 0
total = 0
for sequence in attack_sequences:
    sequence = sequence.split(' ')[:-1]
    j=3
    while j<=len(sequence):
        if tuple(sequence[j-3:j]) == ('Scan','Sensitive_URI','New_Kernel_Module'):
            countc += 1
        elif tuple(sequence[j-3:j]) == ('Sensitive_URI','Sensitive_URI','Sensitive_URI'):
            countr += 1
        total += 1
        j+=1

probsc, probsr = [0,0,0,0,0,countc/total,0,0,0,0,0], [0,0,0,0,countr/total,0,0,0,0,0,0]
print(countc,countr,total)

200 186 2798


### Task 3.2  You will have to submit the graph you draw through Compass

### Task 3.3

In [11]:
### Method 1: Keep the original graph, and create 5 subgraphs

G1 = FactorGraph() 
f_1 = DiscreteFactor(['e1','s1'],[1,11],FACTOR_FUNCTION_MAP['Scan'])
f_3 = DiscreteFactor(['e3','s3'],[1,11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
f_4 = DiscreteFactor(['e4','s4'],[1,11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
f_5 = DiscreteFactor(['e5','s5'],[1,11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
f_6 = DiscreteFactor(['e6','s6'],[1,11],FACTOR_FUNCTION_MAP['New_Kernel_Module'])

r = DiscreteFactor(['e3','e4','e5','s5'],[1,1,1,11],probsr)
c = DiscreteFactor(['e1','e3','e6','s6'],[1,1,1,11],probsc)

nodes1 = ['e1','e3','e4','e5','e6','s1','s3','s4','s5','s6',]
G1.add_nodes_from(nodes1) 
G1.add_factors(f_1, f_3, f_4, f_5, f_6, r, c)

G1.add_edges_from([('e1',c), ('e3',c), ('e6',c), ('s6',c),
                  ('e3',r), ('e4',r), ('e5',r), ('s5',r),
                  ('e1',f_1), ('s1',f_1), ('e3',f_3), ('s3',f_3), 
                  ('e4',f_4), ('s4',f_4), ('e5',f_5), ('s5',f_5), 
                  ('e6',f_6), ('s6',f_6),])

bp1 = BeliefPropagation(G1)


G2 = FactorGraph()
f_2 = DiscreteFactor(['e2','s2'],[1,11],FACTOR_FUNCTION_MAP['Login'])
nodes2 = ['e2','s2']
G2.add_nodes_from(nodes2)  ## Add random variables 
G2.add_factors(f_2)
G2.add_edges_from([('e2',f_2), ('s2',f_2)])
bp2 = BeliefPropagation(G2)

G3 = FactorGraph()
f_7 = DiscreteFactor(['e7','s7'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes3 = ['e7','s7']
G3.add_nodes_from(nodes3)  ## Add random variables 
G3.add_factors(f_7)
G3.add_edges_from([('e7',f_7), ('s7',f_7)])
bp3 = BeliefPropagation(G3)

G4 = FactorGraph()
f_8 = DiscreteFactor(['e8','s8'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes4 = ['e8','s8']
G4.add_nodes_from(nodes4)  ## Add random variables 
G4.add_factors(f_8)
G4.add_edges_from([('e8',f_8), ('s8',f_8)])
bp4 = BeliefPropagation(G4)

G5 = FactorGraph()
f_9 = DiscreteFactor(['e9','s9'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes5 = ['e9','s9']
G5.add_nodes_from(nodes5)  ## Add random variables 
G5.add_factors(f_9)
G5.add_edges_from([('e9',f_9), ('s9',f_9)])
bp5 = BeliefPropagation(G5)

P_s1 = bp1.query(['s1'])
P_s2 = bp2.query(['s2'])
P_s3 = bp1.query(['s3'])
P_s4 = bp1.query(['s4'])
P_s5 = bp1.query(['s5'])
P_s6 = bp1.query(['s6'])
P_s7 = bp3.query(['s7'])
P_s8 = bp4.query(['s8'])
P_s9 = bp5.query(['s9'])
P_s1.normalize()
P_s2.normalize()
P_s3.normalize()
P_s4.normalize()
P_s5.normalize()
P_s6.normalize()
P_s7.normalize()
P_s8.normalize()
P_s9.normalize()
P_s1,P_s2,P_s3,P_s4,P_s5,P_s6,P_s7,P_s8,P_s9 = P_s1.values,P_s2.values,P_s3.values,P_s4.values,P_s5.values,P_s6.values,P_s7.values,P_s8.values,P_s9.values

Eliminating: e9: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.63it/s]


In [12]:
### Method 2: Prune the original factor graph, and create 9 subgraphs

G1 = FactorGraph()
f_1 = DiscreteFactor(['s1'],[11],FACTOR_FUNCTION_MAP['Scan'])
nodes1 = ['s1']
G1.add_nodes_from(nodes1)  ## Add random variables 
G1.add_factors(f_1)
G1.add_edges_from([('s1',f_1)])
bp1 = BeliefPropagation(G1)

G2 = FactorGraph()
f_2 = DiscreteFactor(['e2','s2'],[1,11],FACTOR_FUNCTION_MAP['Login'])
nodes2 = ['e2','s2']
G2.add_nodes_from(nodes2)  ## Add random variables 
G2.add_factors(f_2)
G2.add_edges_from([('e2',f_2), ('s2',f_2)])
bp2 = BeliefPropagation(G2)

G3 = FactorGraph()
f_3 = DiscreteFactor(['s3'],[11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
nodes3 = ['s3']
G3.add_nodes_from(nodes3)  ## Add random variables 
G3.add_factors(f_3)
G3.add_edges_from([('s3',f_3)])
bp3 = BeliefPropagation(G3)

G4 = FactorGraph()
f_4 = DiscreteFactor(['s4'],[11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
nodes4 = ['s4']
G4.add_nodes_from(nodes4)  ## Add random variables 
G4.add_factors(f_4)
G4.add_edges_from([('s4',f_4)])
bp4 = BeliefPropagation(G4)

G5 = FactorGraph()
f_5 = DiscreteFactor(['s5'],[11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
r = DiscreteFactor(['s5'],[11],probsr)
nodes5 = ['s5']
G5.add_nodes_from(nodes5)  ## Add random variables 
G5.add_factors(f_5, r)
G5.add_edges_from([('s5',r), ('s5',f_5)])
bp5 = BeliefPropagation(G5)

G6 = FactorGraph()
f_6 = DiscreteFactor(['s6'],[11],FACTOR_FUNCTION_MAP['Sensitive_URI'])
c = DiscreteFactor(['s6'],[11],probsr)
nodes6 = ['s6']
G6.add_nodes_from(nodes6)  ## Add random variables 
G6.add_factors(f_6, c)
G6.add_edges_from([('s6',c), ('s6',f_6)])
bp6 = BeliefPropagation(G6)

G7 = FactorGraph()
f_7 = DiscreteFactor(['e7','s7'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes7 = ['e7','s7']
G7.add_nodes_from(nodes7)  ## Add random variables 
G7.add_factors(f_7)
G7.add_edges_from([('e7',f_7), ('s7',f_7)])
bp7 = BeliefPropagation(G7)

G8 = FactorGraph()
f_8 = DiscreteFactor(['e8','s8'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes8 = ['e8','s8']
G8.add_nodes_from(nodes8)  ## Add random variables 
G8.add_factors(f_8)
G8.add_edges_from([('e8',f_8), ('s8',f_8)])
bp8 = BeliefPropagation(G8)

G9 = FactorGraph()
f_9 = DiscreteFactor(['e9','s9'],[1,11],FACTOR_FUNCTION_MAP['DNS_Tunneling'])
nodes9 = ['e9','s9']
G9.add_nodes_from(nodes9)  ## Add random variables 
G9.add_factors(f_9)
G9.add_edges_from([('e9',f_9), ('s9',f_9)])
bp9 = BeliefPropagation(G9)

P_s1 = bp1.query(['s1'])
P_s2 = bp2.query(['s2'])
P_s3 = bp3.query(['s3'])
P_s4 = bp4.query(['s4'])
P_s5 = bp5.query(['s5'])
P_s6 = bp6.query(['s6'])
P_s7 = bp7.query(['s7'])
P_s8 = bp8.query(['s8'])
P_s9 = bp9.query(['s9'])
P_s1.normalize()
P_s2.normalize()
P_s3.normalize()
P_s4.normalize()
P_s5.normalize()
P_s6.normalize()
P_s7.normalize()
P_s8.normalize()
P_s9.normalize()
P_s1,P_s2,P_s3,P_s4,P_s5,P_s6,P_s7,P_s8,P_s9 = P_s1.values,P_s2.values,P_s3.values,P_s4.values,P_s5.values,P_s6.values,P_s7.values,P_s8.values,P_s9.values

0it [00:00, ?it/s]
Eliminating: e2: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.87it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
Eliminating: e9: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.69it/s]


### Task 3.4

#### a. At every time point, provide the marginal probability of each state (Since we have 9 time points and 11 possible states, you should provide 99 probability values here)

In [13]:
import pandas as pd
pd.DataFrame([P_s1,P_s2, P_s3, P_s4, P_s5, P_s6,P_s7, P_s8, P_s9])

,0,1,2,3,4,5,6,7,8,9,10
0,0.936000,0.064,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
1,1.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
2,0.553333,0.000,0.0,0.0,0.446667,0.0,0.0,0.0,0.00,0.0,0.0
3,0.553333,0.000,0.0,0.0,0.446667,0.0,0.0,0.0,0.00,0.0,0.0
4,0.000000,0.000,0.0,0.0,1.000000,0.0,0.0,0.0,0.00,0.0,0.0
5,0.000000,0.000,0.0,0.0,1.000000,0.0,0.0,0.0,0.00,0.0,0.0
6,0.020000,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.98,0.0,0.0
7,0.020000,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.98,0.0,0.0
8,0.020000,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.98,0.0,0.0


#### b. At every time point, provide the most probable state

In [14]:
print("The most probable states for s1-s9 are benign, benign, benign, benign, privilege_escalation, persistence, exfiltration, exfiltration, exfiltration")

The most probable states for s1-s9 are benign, benign, benign, benign, privilege_escalation, persistence, exfiltration, exfiltration, exfiltration


### Task 3.5

In [15]:
ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}
print("The recommended actions are No-Op, No-Op, No-Op, No-Op, Monitor, Monitor, Stop Attack, Stop Attack, Stop Attack.")

The recommended actions are No-Op, No-Op, No-Op, No-Op, Monitor, Monitor, Stop Attack, Stop Attack, Stop Attack.


### Task 3.6

#### Indicate the earliest stage in which your model should recommend stopping the attack

In [16]:
print("The earliest stage to stop the attack is s7.")

The earliest stage to stop the attack is s7.


### Task 3.7

#### a. Judge whether the most probable states for $s_1-s_6,s_8,s_9$ remain the same as Task3.2
#### b. State the reason for your judgement

In [17]:
print ("The most probable states should remain the same as Task 3.2 because the subgraph e7 and s7 are disconnected to the rest of the graph.")

The most probable states should remain the same as Task 3.2 because the subgraph e7 and s7 are disconnected to the rest of the graph.


### Task 3.8

#### a. Draw an HMM model for the attack scenario given the provided states and events.
#### b. What parameters are needed for this HMM model to work?
#### c. Give an example of an advantage of the FG over the HMM model.

In [18]:
print("3.8a See drawing in the slides.")
print("3.8b The transition probability matrix, the observation matrix, and the initial distribution for the hidden state: P(S1), P(S2|S1), P(S3|S2), P(S4|S3), P(S5|S4), P(S6|S5), P(S7|S6), P(S8|S7), P(S9|S8), P(E1|S1), P(E2|S2), P(E3|S3), P(E4|S4), P(E5|S5), P(E6|S6), P(E7|S7), P(E8|S8), P(E9|S9).")
print("3.8c Give an example of an advantage of the FG over the HMM model. FG is a generalization of the HMM model. One of its advantages over HMM is that it can model the relationship between multiple evidences and an underlying state. (E.g. links between the repetitive events and the attack stage) This is impossible for HMM because HMM assumes that the hidden states have a temporal order, which does not apply to many situations.")

3.8a See drawing in the slides.
3.8b The transition probability matrix, the observation matrix, and the initial distribution for the hidden state: P(S1), P(S2|S1), P(S3|S2), P(S4|S3), P(S5|S4), P(S6|S5), P(S7|S6), P(S8|S7), P(S9|S8), P(E1|S1), P(E2|S2), P(E3|S3), P(E4|S4), P(E5|S5), P(E6|S6), P(E7|S7), P(E8|S8), P(E9|S9).
3.8c Give an example of an advantage of the FG over the HMM model. FG is a generalization of the HMM model. One of its advantages over HMM is that it can model the relationship between multiple evidences and an underlying state. (E.g. links between the repetitive events and the attack stage) This is impossible for HMM because HMM assumes that the hidden states have a temporal order, which does not apply to many situations.
